## Deteccion del tablero en la imagen

-----
Se usara el tablero B para esta seccion.

### Representaciones:
<pre>
Se usa una representacion de tablero mediante un array de 64 elementos de 8 bits cada uno. En cada uno de esos elementos se
 almacenara una pieza. Aqui se presentan los encodings para cada una de ellas. 
    * Los primeros 4 bits se usaran para representar el color.
        - 0000 XXXX : Negro
        - 1111 XXXX : Blanco
    * Con los 4 bits restantes se representa la pieza:
         Pieza        Codificacion
        - Peon      | 0000
        - Caballo   | 0001
        - Alfil     | 0010
        - Torre     | 0011
        - Reina     | 0100
        - Rey       | 0101
    * Un casillero vacio se representa mediante: 1111 1111
</pre>
----------------------------------------
### Detalle del algoritmo:
<pre>
El algoritmo consta de 4 partes:
        
    1. Deteccion del tablero: Cuando la foto se captura el tablero puede estar torcido y puesto en cualquier tipo de posicion. Por 
    lo que lo primero es identificarlo, recortarlo, centrarlo y alinearlo.

    2. Luego se reconocen cada una de las casillas individuales. Se determinan sus dimensiones, mediante 2 puntos, la esquina 
    superior izquierda y la esquina inferior izquierda.

    3. Se aplica 2 mascaras. Una de ellas detecta piezas blancas, la otra negras. Si se detecta una pieza se almacena en un array de 
    64 elementos booleanos.

    4. Se compara el array de booleanos con el de piezas y se realizan las debidas actualizaciones.  

A tener en cuenta: 

Para el correcto funcionamiento del algoritmo se debe conocer la posicion inicial de las piezas, ya que en base a estas se realiza 
la deduccion de posiciones despues.
</pre>

In [1]:
import cv2
import numpy as np

In [2]:
# Cargo las imagenes del tablero b
b_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial.jpeg")
b_pos_inicio2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial-2.jpeg")
mov1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-1.jpg")
mov2= cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-2.jpg")
mov3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-3.jpg")
mov4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-4.jpg")
mov5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-5.jpg")
mov6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-6.jpg")

In [3]:
# Funciones comunes 
def showImg(img, text='image'):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

# 1. Deteccion y recorte del tablero

In [4]:
# Trabajaremos con esta posicion.
showImg(b_pos_inicio, "Posicion inicial")

#### Rotamos y recortamos el tablero

In [5]:
def rotarRecortarImagen(img, verbose=False):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if (verbose): showImg(img_gray, 'Imagen en blanco y negro')

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrarContornos(img_gray, 100, 300, False)

    if (verbose): showImg(edges, 'Contornos detectados')

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    if (verbose): showImg(rotated, 'Imagen rotada')

    #  Convierto la imagen a hsv
    hsv_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)


    if (verbose): showImg(hsv_image, 'HSV aplicado a la imagen rotada')

    # Defino el rango de colores para poder recortar el tablero, en este caso se usa teniendo en cuenta el fondo de color blanco
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([255, 90, 200])

    # Creo una mascara que identifique aproximadamente el color #778078, este es el color del fondo.
    mask = cv2.inRange(hsv_image, lower_color, upper_color)

    if (verbose): showImg(mask, 'Mascara, hsv, imagen rotada')

    # Se encuentran los contornos
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    booleano = True

    for c in contorno:
        area = cv2.contourArea(c)
        if (area > 40000):
            cont = cont + 1
            x,y,w,h = cv2.boundingRect(c)
            # Recortamos el contorno encontrado.
            if (booleano):
                # rotamos la imagen encontrada
                cropped_image = rotated[y:(y+h), x:(x+w)].copy()
                # recorto el borde rojo del tablero 
                cropped_image = cropped_image[60:843, 55:830]
                booleano = False
            cv2.rectangle(rotated,(x,y),(x+w,y+h),(0,255,255),2)
            cv2.putText(rotated,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)
            area = cv2.contourArea(c)
    
    if (verbose): showImg(cropped_image, 'Imagen rotada y recortada')

    return cropped_image

### Tests

In [6]:
#cropped_image = rotarRecortarImagen(b_pos_inicio) # no funciona
#showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(b_pos_inicio2) # no funciona
#showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov1, True) # funciona
showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(mov2) # no funciona
#showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(mov3) # funciona
#showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(mov4) # funciona
#showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(mov5) # funciona
#showImg(cropped_image, 'cropped_image')
#cropped_image = rotarRecortarImagen(mov6) # funciona
#showImg(cropped_image, 'cropped_image')


-------
### Algunas pruebas de deteccion automatica de casilleros


In [6]:
def dibujarContornos(img, masked_img, title, bool):
    retorno = img.copy()
    edges = cv2.Canny(masked_img, 200, 500).copy()
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chessboard_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if (area > 1000):
            chessboard_contours.append(contour)
    for contour in chessboard_contours:
        cv2.drawContours(retorno , [contour], -1, (0, 255, 0), 2).copy()
    if bool:
        showImg(retorno , title)
    return retorno 


image = rotarRecortarImagen(mov3) 

# Mascara
lower_black = np.array([0, 0, 0])
upper_black = np.array([50, 50, 50])

black_mask = cv2.inRange(image, lower_black, upper_black)
contours, _ = cv2.findContours(black_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

grid_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 900:
        grid_contours.append(contour)
    

cv2.drawContours(image, grid_contours, -1, (0, 255, 0), 2)

showImg(image, 'black_mask')

-------
### Deteccion de piezas en casilleros.

In [7]:
# mascara para casillero blanco pieza blanca
lower_cblanco_pblanco = np.array([100,100,50],np.uint8)
upper_cblanco_pblanco = np.array([170,150,150],np.uint8)

# mascara para casillero negro con pieza negra
lower_cnegro_pnegra = np.array([30,30,30],np.uint8)
upper_cnegro_pnegra = np.array([100,50, 50],np.uint8)

# mascara para casillero negro con pieza blanca
lower_cnegro_pblanca = np.array([0, 0, 0])
upper_cnegro_pblanca = np.array([50, 50, 50])

# mascara para casillero blanco con pieza negra
lower_cblanco_pnegra = np.array([0, 0, 0])
upper_cblanco_pnegra = np.array([50, 50, 50])

lower_mask1 = np.array([100,100,50],np.uint8)
upper_mask1 = np.array([170,150,150],np.uint8)

lower_mask2 = np.array([30,30,30],np.uint8)
upper_mask2 = np.array([100,50, 50],np.uint8)

lower_mask3 = np.array([0, 0, 0])
upper_mask3 = np.array([50, 50, 50])

In [10]:
def encontrarContornosPieza(image, mask):
    retorno = image.copy()
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    grid_contours = []   
    for contour in contorno:
        if 6000 > cv2.contourArea(contour) > 10:
            grid_contours.append(contour)

    cv2.drawContours(retorno, grid_contours, -1, (0, 255, 0), 2)
    return retorno, len(contorno)

def detectarPieza(casilla1, verbose=False):
    mascara1 = cv2.inRange(casilla1, lower_mask1, upper_mask1).copy()
    mascara2 = cv2.inRange(casilla1, lower_mask2, upper_mask2).copy()
    mascara3 = cv2.inRange(casilla1, lower_mask3, upper_mask3).copy()
    
    img1, contorno1 = encontrarContornosPieza(casilla1,mascara1)
    img2, contorno2 = encontrarContornosPieza(casilla1,mascara2)
    img3, contorno3 = encontrarContornosPieza(casilla1,mascara3)
    if verbose:
        showImg(casilla1)
        showImg(img1, 'img1')
        showImg(img2, 'img2')
        showImg(img3, 'img3')
        print(contorno1,contorno2,contorno3)
    if contorno1 > 0 and contorno2 > 0 and contorno3 > 0:
        if verbose: print('Pieza detectada')
        return '1'
    else:
        if verbose: print('Casillero vacio')
        return '.'        

def recortarCasillero(image, i, j):
    start_y = int(100*i)
    end_y = int(97*i+85)
    start_x = int(97*j)
    end_x = int(97*j+85)
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero

image = rotarRecortarImagen(mov3) 
detectarPieza(recortarCasillero(image, 1,0))

#Procesar todo el tablero:
matriz = np.empty((8, 8), dtype=str)
for i in range(8):
    for j in range(8):
        cropped_image = recortarCasillero(image, i,j)
        matriz[i][j] = detectarPieza(cropped_image)
print(matriz)     

cropped_image = recortarCasillero(image, 3,0)
detectarPieza(cropped_image, True)
showImg(cropped_image)


[['1' '1' '1' '1' '.' '1' '.' '1']
 ['1' '1' '1' '.' '.' '.' '1' '.']
 ['.' '.' '.' '.' '.' '1' '.' '.']
 ['1' '.' '.' '.' '1' '.' '.' '.']
 ['.' '.' '.' '.' '1' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['1' '1' '1' '1' '.' '1' '1' '1']
 ['1' '1' '1' '1' '1' '1' '1' '1']]


In [ ]:
# TODO: Ver como se puede utilizar la informacion del heatmap, esta interesante ...
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv_image)
heatmap = cv2.applyColorMap(v, cv2.COLORMAP_HSV)

showImg(heatmap, 'heatmap')

-----------------------
# Tablero B

In [ ]:
 # Carga las imagenes en una variable
a_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio.jpeg")
pos_mid = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid.jpeg")
pos_mid2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2.jpeg")
pos_ataque = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-ataque.jpeg")
iluminada = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/iluminada.jpeg")
pos_inicial_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio-esp.jpeg")
pos_mid_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid-esp.jpeg")
pos_mid2_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2-esp.jpeg")
tablero_normal = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-ideal/tablero-comun.png')

In [ ]:
# procesamos la imagen 
cv2.imshow('Pos mid', pos_mid)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(pos_mid, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
cv2.imshow('tablero normal: ',tablero_normal)
cv2.waitKey(10000)
cv2.destroyAllWindows()

ret, corners = cv2.findChessboardCorners(pos_mid_gray, (9, 9))

print(ret, corners)

# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid, (9, 9), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

False None


In [10]:
import cv2
import numpy as np


def nothing(x):
    pass

# Create a black image, a window
img = np.zeros((300, 512, 3), np.uint8)
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('R', 'image', 0, 255, nothing)
cv2.createTrackbar('G', 'image', 0, 255, nothing)
cv2.createTrackbar('B', 'image', 0, 255, nothing)
# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image', 0, 1, nothing)
while (1):
    cv2.imshow('image', img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    # get current positions of four trackbars
    r = cv2.getTrackbarPos('R', 'image')
    g = cv2.getTrackbarPos('G', 'image')
    b = cv2.getTrackbarPos('B', 'image')
    s = cv2.getTrackbarPos(switch, 'image')
    if s == 0:
        img[:] = 0
    else:
        img[:] = [b, g, r]
cv2.destroyAllWindows()


KeyboardInterrupt: 